In [3]:
import pandas as pd
import numpy as np
from common import Data

In [11]:
df = Data.get_nwsl_matches()
df

,date,time,season,home,away,home_score,away_score,home_penalty_score,away_penalty_score,home_xg,away_xg,referee,attendance,stadium,is_forfeit,is_extra_time,is_pks,is_playoffs
0,2013-04-13,19:35,2013,KC,POR,1,1,0.0,0.0,NaN,NaN,Kari Seitz,6784.0,Shawnee Mission District Stadium,False,False,False,False
1,2013-04-14,17:00,2013,CHI,RGN,1,1,0.0,0.0,NaN,NaN,Josh Wilkens,1255.0,Village of Lisle-Benedictine University ...,False,False,False,False
2,2013-04-14,18:00,2013,NJNY,WNY,1,0,0.0,0.0,NaN,NaN,John McCloskey,2611.0,Yurcak Field,False,False,False,False
3,2013-04-14,18:30,2013,BOS,WAS,1,1,0.0,0.0,NaN,NaN,Hernan Aguilar,2634.0,Dilboy Stadium,False,False,False,False
4,2013-04-20,19:00,2013,WAS,WNY,1,1,0.0,0.0,NaN,NaN,Kari Seitz,4569.0,Maureen Hendricks Field at Maryland Socc...,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,2023-10-15,17:00,2023,ORL,HOU,1,0,0.0,0.0,2.7,1.1,Natalie Simon,8504.0,Exploria Stadium,False,False,False,False
1275,2023-10-15,17:00,2023,WAS,NC,0,1,0.0,0.0,1.0,1.2,Ricardo Fierro,15479.0,Audi Field,False,False,False,False
1276,2023-10-15,17:00,2023,NJNY,KCC,2,2,0.0,0.0,1.3,0.7,Alyssa Nichols,9005.0,Red Bull Arena,False,False,False,False
1277,2023-10-20,19:00,2023,RGN,LA,1,0,0.0,0.0,0.5,0.2,Brad Jensen,12152.0,Lumen Field,False,False,False,True


In [14]:
def calc_result(row, team):
    opp = "away" if team == "home" else "home"
    if row[team + "_score"] > row[opp + "_score"]:
        return "W"
    elif row[team + "_score"] < row[opp + "_score"]:
        return "L"
    elif not row["is_playoffs"]:
        return "D"
    elif row[team + "_penalty_score"] > row[opp + "_penalty_score"]:
        return "W"
    elif row[team + "_penalty_score"] < row[opp + "_penalty_score"]:
        return "L"
    else:
        print(row)
        raise Exception("Can't calculate score")


df["home_result"] = df.apply(lambda r: calc_result(r, "home"), axis=1)
df["away_result"] = df.apply(lambda r: calc_result(r, "away"), axis=1)

df = df[["date", "home", "away", "home_result", "away_result"]]
df

,date,home,away,home_result,away_result
0,2013-04-13,KC,POR,D,D
1,2013-04-14,CHI,RGN,D,D
2,2013-04-14,NJNY,WNY,W,L
3,2013-04-14,BOS,WAS,D,D
4,2013-04-20,WAS,WNY,D,D
...,...,...,...,...,...
1274,2023-10-15,ORL,HOU,W,L
1275,2023-10-15,WAS,NC,L,W
1276,2023-10-15,NJNY,KCC,D,D
1277,2023-10-20,RGN,LA,W,L


In [17]:
df2 = df.rename(columns={"home": "team", "away": "opponent", "home_result": "result"}).append(
    df.rename(columns={"away": "team", "home": "opponent", "away_result": "result"})
)[["date", "team", "opponent", "result"]]
df2

,date,team,opponent,result
0,2013-04-13,KC,POR,D
1,2013-04-14,CHI,RGN,D
2,2013-04-14,NJNY,WNY,W
3,2013-04-14,BOS,WAS,D
4,2013-04-20,WAS,WNY,D
...,...,...,...,...
1274,2023-10-15,HOU,ORL,L
1275,2023-10-15,NC,WAS,W
1276,2023-10-15,KCC,NJNY,D
1277,2023-10-20,LA,RGN,L


In [27]:
df3 = df2.groupby(["team", "opponent", "result"]).agg(count=("date", "count")).reset_index()
df3

,team,opponent,result,count
0,BOS,CHI,D,3
1,BOS,CHI,L,8
2,BOS,CHI,W,1
3,BOS,HOU,D,1
4,BOS,HOU,L,7
...,...,...,...,...
549,WNY,RGN,L,5
550,WNY,RGN,W,2
551,WNY,WAS,D,3
552,WNY,WAS,L,3


In [29]:
df3[(df3["team"] == "SD") & (df3["opponent"] == "RGN")]

,team,opponent,result,count
457,SD,RGN,D,2
458,SD,RGN,L,6


In [31]:
df3[(df3["team"] == "POR") & (df3["opponent"] == "NJNY")]

,team,opponent,result,count
372,POR,NJNY,D,7
373,POR,NJNY,L,7
374,POR,NJNY,W,13


In [32]:
df3[(df3["team"] == "SD") & (df3["opponent"] == "POR")]

,team,opponent,result,count
454,SD,POR,D,2
455,SD,POR,L,4
456,SD,POR,W,3


In [33]:
df3[(df3["team"] == "SD") & (df3["opponent"] == "NJNY")]

,team,opponent,result,count
450,SD,NJNY,W,4


In [35]:
df3[(df3["team"] == "POR") & (df3["opponent"] == "RGN")]

,team,opponent,result,count
378,POR,RGN,D,9
379,POR,RGN,L,15
380,POR,RGN,W,14


In [36]:
df3[(df3["team"] == "RGN") & (df3["opponent"] == "NJNY")]

,team,opponent,result,count
416,RGN,NJNY,D,7
417,RGN,NJNY,L,8
418,RGN,NJNY,W,12
